# Added:

- LSTM layer and linear layer





# Install packages

In [ ]:
pip install gym
pip install yfinance
pip install numpy --upgrade #check if numpy is the most recent version

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pathSP = "/content/drive/MyDrive/Colab Notebooks/Thesis code/data/SP2014-2016.csv"

In [ ]:
pathMSFT = "/content/drive/MyDrive/Colab Notebooks/Thesis code/data/MSFT2014-2016.csv"

# Trading_env

In [ ]:
import random
from statistics import mean
import warnings
warnings.filterwarnings("ignore")

class TradingSystem_v0:
    def __init__(self, returns_data, k_value, action_dim, initial_balance,mode,stock):
        self.mode = mode  # test or train
        self.initial_balance = initial_balance
        self.balance = self.initial_balance
        self.k = k_value
        self.tickers = stock
        self.curr_position = 0
        self.action_space_dim = action_dim
        self.total_steps = returns_data.shape[0] - self.k
        self.current_step = 0
        self.reward = 0.0
        self.core = float(returns_data.iloc[returns_data.shape[0]-1,3])
        self.dev = float(np.std(returns_data.iloc[:,3]))
        self.trans_cost = 0.0005 # transaction cost
        self.slippage_rate = 0.001 # slippage rate
        for i in range(0,returns_data.shape[1]):
            col = np.asarray(returns_data.iloc[:,i])
            norm_value = preprocessing.normalize([col]).squeeze()
            for j in range(0,returns_data.shape[0]):
                returns_data.iloc[j,i] = norm_value[j]
        returns_data = returns_data.assign(MA = MA(returns_data.iloc[:,3],10), MACD = MACD(returns_data.iloc[:,3],10,20)
        ,ROC = ROC(returns_data.iloc[:,3],10), SR = SR(returns_data.iloc[:,3],10))
        self.initial_state = returns_data.iloc[0,:]
        self.state = self.initial_state
        self.batch = 16
        self.r_ts = returns_data
        self.is_terminal = False

    # write step function that returns obs(next state), reward, is_done
    def step(self, action):
        self.current_step += 1
        if self.current_step == self.total_steps:
            self.is_terminal = True

        self.pos_change = action - self.action_space_dim//2  # get action(position) of the actor

        self.curr_position = self.curr_position + self.pos_change

        if self.pos_change == 0:

            close_price = self.r_ts.iloc[self.current_step,3]
            close_price_bf = self.r_ts.iloc[self.current_step-1,3]
            self.reward =  self.pos_change * (close_price - close_price_bf)
        else:
            close_price = self.r_ts.iloc[self.current_step,3]
            close_price_bf = self.r_ts.iloc[self.current_step-1,3]
            self.reward =  self.pos_change * (close_price - close_price_bf)  - abs(self.pos_change)* (self.trans_cost+self.slippage_rate)

        self.state = self.r_ts.iloc[self.current_step+1,:]

        return self.state, self.reward, self.is_terminal

    def reset(self):
        self.r_ts = self.r_ts
        self.total_steps = len(self.r_ts) - self.k
        self.balance = self.initial_balance
        self.current_step = 0
        self.initial_state = self.r_ts.iloc[0,:]
        self.state = self.initial_state
        self.reward = 0.0
        self.is_terminal = False
        return self.state



# dqn

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import math
import numpy as np


class MLP(nn.Module):
    def __init__(self, state_dim, action_dim, hidden_dim=128):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(state_dim, 64)  # input layer
        self.fc2 = nn.Linear(64, hidden_dim)  # hidden layer 1
        self.fc3 = nn.Linear(hidden_dim, 128)  # hidden layer 2
        self.lstm_cells = nn.LSTMCell(128, 128) # lstm layer (returns hidden state and cell state)
        self.fc4 = nn.Linear(128, action_dim) # connects lstm layer to linear layer

    def forward(self, x):
        # activation function
        x = torch.nn.functional.relu(self.fc1(x))
        x = torch.nn.functional.relu(self.fc2(x))
        x = torch.nn.functional.relu(self.fc3(x))
        hx,cx = self.lstm_cells(x)
        # x = hx.clone().detach()
        # x = torch.tensor(hx) (gives warning)


        return self.fc4(hx)


class ReplayBuffer:
    def __init__(self, capacity):
        self.capacity = capacity  # capacity of buffer
        self.buffer = []  # replay buffer
        self.position = 0

    def push(self, state, action, reward, next_state, done):
        ''' replay buffer is a queue (LIFO)
        '''
        if len(self.buffer) < self.capacity:
            self.buffer.append(None)
        self.buffer[self.position] = (state, action, reward, next_state, done)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        state, action, reward, next_state, done = zip(*batch)
        return state, action, reward, next_state, done

    def __len__(self):
        return len(self.buffer)


class DQN:
    def __init__(self, state_dim, action_dim, cfg):

        self.action_dim = action_dim
        self.device = cfg.device  # cpu or gpu
        self.gamma = cfg.gamma  # discount factor
        self.frame_idx = 0  # attenuation
        self.epsilon = lambda frame_idx: cfg.epsilon_end + \
                                         (cfg.epsilon_start - cfg.epsilon_end) * \
                                         math.exp(-1. * frame_idx / cfg.epsilon_decay)
        self.batch_size = cfg.batch_size
        self.policy_net = MLP(state_dim, action_dim, hidden_dim=cfg.hidden_dim).to(self.device)
        self.target_net = MLP(state_dim, action_dim, hidden_dim=cfg.hidden_dim).to(self.device)
        for target_param, param in zip(self.target_net.parameters(),
                                       self.policy_net.parameters()):  # copy parameters to target net
            target_param.data.copy_(param.data)
        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=cfg.lr) # optimizer
        self.memory = ReplayBuffer(cfg.memory_capacity)  # experience replay

    def choose_action(self, state):
        self.frame_idx += 1
        if random.random() > self.epsilon(self.frame_idx):
            with torch.no_grad():
                state = torch.tensor([state], device=self.device, dtype=torch.float32)   # received observation
                q_values = self.policy_net(state)
                action = q_values.max(1)[1].item()  # choose the action with maximum q-value
        else:
            action = random.randrange(self.action_dim)
        return action

    def update(self):
        if len(self.memory) < self.batch_size:
            return
        state_batch, action_batch, reward_batch, next_state_batch, done_batch = self.memory.sample(
            self.batch_size)
        # transfer to tensor
        state_batch = torch.tensor(state_batch, device=self.device, dtype=torch.float)
        action_batch = torch.tensor(action_batch, device=self.device).unsqueeze(1)
        reward_batch = torch.tensor(reward_batch, device=self.device, dtype=torch.float)
        next_state_batch = torch.tensor(next_state_batch, device=self.device, dtype=torch.float)
        done_batch = torch.tensor(np.float32(done_batch), device=self.device)
        q_values = self.policy_net(state_batch).gather(dim=1, index=action_batch)
        next_q_values = self.target_net(next_state_batch).max(1)[0].detach()
        # calculate the expected q-value, for final state, done_batch[0]=1 and the corresponding
        # expected_q_value equals to reward
        expected_q_values = reward_batch + self.gamma * next_q_values * (1 - done_batch)
        loss = nn.MSELoss()(q_values, expected_q_values.unsqueeze(1))
        # update the network
        self.optimizer.zero_grad()
        loss.backward()
        for param in self.policy_net.parameters():  # avoid gradient explosion by using clip
            param.grad.data.clamp_(-1, 1)
        self.optimizer.step()

    def save(self, path):
        torch.save(self.target_net.state_dict(), path + 'dqn_checkpoint.pth')

    def load(self, path):
        self.target_net.load_state_dict(torch.load(path + 'dqn_checkpoint.pth'))
        for target_param, param in zip(self.target_net.parameters(), self.policy_net.parameters()):
            param.data.copy_(target_param.data)



# Main

In [ ]:
import sys
import os
import os.path


curr_path = os.path.abspath('')

import gym
import torch
import numpy as np
import random
import yfinance as yf
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt

curr_time = dt.datetime.now().strftime("%Y%m%d-%H%M%S")
random.seed(11)



class Config:
    '''
    hyperparameters
    '''

    def __init__(self):
        ################################## env hyperparameters ###################################
        self.algo_name = 'DQN' # algorithmic name
        self.env_name = 'TradingSystem_v0' # environment name
        self.device = torch.device(
            "cuda" if torch.cuda.is_available() else "cpu")  # examine GPU
        self.seed = 11 # random seed
        self.train_eps = 100 # number of training episode
        self.state_space_dim = 8 # state space size (K-value)
        self.initial_balance = 1000000
        self.action_space_dim = 3 # action space size (only odd number can be applied)
        ################################################################################

        ################################## algo hyperparameters ###################################
        self.gamma = 0.95  # discount factor
        self.epsilon_start = 0.90  # start epsilon of e-greedy policy
        self.epsilon_end = 0.01  # end epsilon of e-greedy policy
        self.epsilon_decay = 500  # attenuation rate of epsilon in e-greedy policy
        self.lr = 0.0001  # learning rate
        self.memory_capacity = 1000  # capacity of experience replay
        self.batch_size = 64  # size of mini-batch SGD
        self.target_update = 10  # update frequency of target network
        self.hidden_dim = 128  # dimension of hidden layer
        ################################################################################

        ################################# save path ##############################
        self.result_path = curr_path + "/outputs/" + self.env_name + \
                           '/' + curr_time + '/results/'
        self.model_path = curr_path + "/outputs/" + self.env_name + \
                          '/' + curr_time + '/models/'
        self.save = True  # whether to save the image
        ################################################################################


def env_agent_config(data, cfg, mode,stock):
    ''' create environment and agent
    '''
    env = TradingSystem_v0(data, cfg.state_space_dim, cfg.action_space_dim, cfg.initial_balance, mode, stock)
    agent = DQN(cfg.state_space_dim, cfg.action_space_dim, cfg)

    if cfg.seed != 0:  # set random seeds
        torch.manual_seed(cfg.seed)
        np.random.seed(cfg.seed)
    return env, agent


def train(cfg, env, agent):
    ''' training
    '''
    print('Start Training!')
    print(f'Environment：{cfg.env_name}, Algorithm：{cfg.algo_name}, Device：{cfg.device}, Stock: {env.tickers}')
    rewards = []  # record total rewards
    ma_rewards = []  # record moving average total rewards
    stocks = env.tickers
    ep_reward = 0
    state = env.reset()
    for i_ep in range(cfg.train_eps):
        ep_reward = 0
        state = env.reset()
        while True:
            action = agent.choose_action(state)
            next_state, reward, done = env.step(action)
            agent.memory.push(state, action, reward, next_state, done)  # save transition
            state = next_state
            agent.update()
            ep_reward +=  reward
            if done:
                break
        if (i_ep + 1) % cfg.target_update == 0:  # update target network
            agent.target_net.load_state_dict(agent.policy_net.state_dict())
        sum_rewards = env.core * env.batch * (1+ep_reward) + env.dev * ep_reward
        rewards.append(sum_rewards)
        if ma_rewards:
            ma_rewards.append(0.9 * ma_rewards[-1] + 0.1 * sum_rewards)
        else:
            ma_rewards.append(sum_rewards)
        if (i_ep + 1) % 10 == 0:
            print('Episode：{}/{}, Reward：{}'.format(i_ep + 1, cfg.train_eps, sum_rewards))
    print('Finish Training!')
    return rewards, ma_rewards


def test(cfg, env, agent):
    print('Start Testing!')
    print(f'Environment：{cfg.env_name}, Algorithm：{cfg.algo_name}, Device：{cfg.device},Stock: {env.tickers}')
    ############# Test does not use e-greedy policy, so we set epsilon to 0 ###############
    cfg.epsilon_start = 0.0
    cfg.epsilon_end = 0.0
    ################################################################################
    stocks = env.tickers
    returns = []
    ep_reward = 0
    state = env.reset()
    while True:
        action = agent.choose_action(state)
        next_state, reward, done = env.step(action)
        state = next_state
        ep_reward += reward
        if done:
            break

    sum_rewards = ep_reward*env.dev + env.core
    returns.append(sum_rewards)
    print(f"Stock：{stocks}，Return：{sum_rewards:.1f}")
    print('Finish Testing!')
    return stocks, sum_rewards


# Running files

# Indicators

In [ ]:
def MA(sorted_data, period):
	ma_list = []
	for i in range(len(sorted_data)):
		if(i < period - 1):
			ma_list.append(sorted_data[i])
			#ma_list.append(None)
		else:
			ma_sum = 0.0
			for j in range(i, i - period, -1):
				ma_sum += sorted_data[j]
			curr_ma = ma_sum / period
			ma_list.append(curr_ma)
	return ma_list

def EMA(data, period):
    sorted_data = np.array([data[i] for i in sorted(data.keys())])
    sma = np.mean(sorted_data[:period])
    ema_list = []
    smooth_weighting = 2.0 / (period + 1)
    for i in range(len(sorted_data)):
        if i < period - 1:
            ema_list.append(sma)
        else:
            new_ema = (sorted_data[i] - ema_list[-1]) * smooth_weighting + ema_list[-1]
            ema_list.append(new_ema)
    assert len(ema_list) == len(sorted_data)
    return np.array(ema_list)

def MACD(sorted_data, period_1, period_2):
	period_1_ema_list = EMA(sorted_data, period_1)
	period_2_ema_list = EMA(sorted_data, period_2)
	assert len(period_1_ema_list) == len(period_2_ema_list)
	macd_list = period_1_ema_list - period_2_ema_list
	return macd_list


def ROC(data, period):
    roc_list = []
    for i in range(len(data)):
        if i == 0:
            roc_list.append(0.0)
        elif i < period:
            curr_roc = (data[i] - data[0]) / data[0]
            roc_list.append(curr_roc)
        else:
            curr_roc = (data[i] - data[i - period]) / data[i - period]
            roc_list.append(curr_roc)
    return roc_list

def SR(sorted_data, period, risk_free_rate=0.007):
    # trading hours per day
    risk_free_rate = risk_free_rate / (6.5 * 60)
    sr_list = []
    for i in range(len(sorted_data)):
        if i < period:
            sr_list.append(0.0)
        else:
            expected_return_list = []
            for j in range(i, i - period, -1):
                perc_change = (sorted_data[j] - sorted_data[j - 1]) / sorted_data[j - 1]
                expected_return_list.append(perc_change)
            expected_return_list = np.array(expected_return_list)
            std = np.std(expected_return_list)
            if std != 0.0:
                sr_list.append((np.mean(expected_return_list) - risk_free_rate) / std)
            else:
                sr_list.append(0.0)
    return sr_list


# Uploading Dataset

In [ ]:
import pandas as pd
from sklearn import preprocessing
import math

### S&P 500 ###
SP = pd.read_csv(r'/Users/gilseyeon/Downloads/data/SP2014-2016.csv')

#SP = pd.read_csv(pathSP)
oo = range(0,len(SP),30)
returns_SP = SP.iloc[oo,].reset_index(drop=True)


returns_SP = returns_SP.iloc[:,2:6] # remove time stamps


#splitting train and test set
splitter = math.ceil(returns_SP.shape[0]*0.9)
SP_train = returns_SP.iloc[0:splitter,:]
SP_test = returns_SP.iloc[splitter+1:,:].reset_index(drop=True)



### Microsoft ###
MSFT = pd.read_csv(r'/Users/gilseyeon/Downloads/data/MSFT2014-2016.csv')
#MSFT = pd.read_csv(pathMSFT)
dd = range(0,len(MSFT),6)
returns_MSFT = MSFT.iloc[dd,].reset_index(drop=True)


returns_MSFT = returns_MSFT.iloc[:,1:5] # remove time stamps


#splitting train and test set
splitter = math.ceil(returns_MSFT.shape[0]*0.9)
MSFT_train = returns_MSFT.iloc[0:splitter,:]
MSFT_test = returns_MSFT.iloc[splitter+1:,:].reset_index(drop=True)


In [ ]:
# Training SP500
cfg = Config()
env, agent = env_agent_config(SP_train, cfg, 'train',"SP500")
rewards_train_SP, ma_rewards_train_SP = train(cfg, env, agent)


Start Training!
Environment：TradingSystem_v0, Algorithm：DQN, Device：cpu, Stock: SP500
Episode：10/100, Reward：21726.97176509192
Episode：20/100, Reward：25033.448567125466
Episode：30/100, Reward：23098.21736353873
Episode：40/100, Reward：26452.34049062942
Episode：50/100, Reward：25129.777057493924
Episode：60/100, Reward：25438.955410266884
Episode：70/100, Reward：24978.65396195801
Episode：80/100, Reward：25235.001330867293
Episode：90/100, Reward：24016.024910429416
Episode：100/100, Reward：25236.06555030652
Finish Training!


In [ ]:
rewards_train_SP


[1310.8527648273762,
 17051.445476931276,
 20353.5016751403,
 20564.11394795461,
 21971.272169949025,
 21481.5525654397,
 21419.894563519505,
 21921.402444894233,
 20815.782025852608,
 21726.97176509192,
 24726.125776451696,
 22649.24030157046,
 23308.680099877987,
 23452.313273575906,
 23303.550016427307,
 23506.71275316695,
 23404.35368597615,
 23713.80709854263,
 23203.674127928047,
 25033.448567125466,
 22645.911204863103,
 24934.418596642012,
 25124.180900504216,
 23149.00177155771,
 23767.822367628432,
 24318.244905330434,
 22955.553448160856,
 24059.563894204657,
 22137.936143819155,
 23098.21736353873,
 25637.995524624992,
 25341.04423457853,
 23917.07680297976,
 23048.116238975086,
 23651.81946147305,
 24165.197482746946,
 25434.30340693352,
 24366.886684878427,
 25180.902015733467,
 26452.34049062942,
 25033.57081592259,
 24622.975366600942,
 24571.004490345593,
 23969.375131370252,
 24676.50164049825,
 25590.62425936139,
 25236.147413340324,
 25993.82583947135,
 24677.129257

In [ ]:
ma_rewards_train_SP


[1310.8527648273762,
 2884.9120360377665,
 4631.77099994802,
 6225.005294748679,
 7799.631982268714,
 9167.824040585812,
 10393.031092879182,
 11545.868228080688,
 12472.85960785788,
 13398.270823581282,
 14531.056318868325,
 15342.874717138538,
 16139.455255412484,
 16870.741057228825,
 17514.021953148673,
 18113.2910331505,
 18642.39729843307,
 19149.538278444026,
 19554.95186339243,
 20102.801533765734,
 20357.11250087547,
 20814.843110452126,
 21245.776889457335,
 21436.09937766737,
 21669.271676663477,
 21934.16899953017,
 22036.307444393242,
 22238.633089374383,
 22228.563394818862,
 22315.52879169085,
 22647.775464984265,
 22917.10234194369,
 23017.099788047297,
 23020.201433140075,
 23083.363235973375,
 23191.546660650733,
 23415.822335279016,
 23510.928770238956,
 23677.92609478841,
 23955.36753437251,
 24063.18786252752,
 24119.166612934863,
 24164.350400675936,
 24144.852873745367,
 24198.017750420655,
 24337.27840131473,
 24427.16530251729,
 24583.831356212697,
 24593.16114

In [ ]:
# Testing SP500
env, agent = env_agent_config(SP_test, cfg, 'test',"SP500")
stocks_test_SP, rewards_test_SP = test(cfg, env, agent)


Start Testing!
Environment：TradingSystem_v0, Algorithm：DQN, Device：cpu,Stock: SP500
Stock：SP500，Return：2039.6
Finish Testing!


In [ ]:
stocks_test_SP

'SP500'

In [ ]:
rewards_test_SP

2039.604905208095

In [ ]:
# Training Microsoft
cfg = Config()
env, agent = env_agent_config(MSFT_train, cfg, 'train','MSFT')
rewards_train_MSFT, ma_rewards_train_MSFT = train(cfg, env, agent)


Start Training!
Environment：TradingSystem_v0, Algorithm：DQN, Device：cpu, Stock: MSFT
Episode：10/100, Reward：730.2911490806887
Episode：20/100, Reward：775.5594298001346
Episode：30/100, Reward：771.5692470785041
Episode：40/100, Reward：752.2763532588352
Episode：50/100, Reward：780.9465252369755
Episode：60/100, Reward：783.1291099589578
Episode：70/100, Reward：761.8604080121287
Episode：80/100, Reward：790.3378611792368
Episode：90/100, Reward：782.6664920104855
Episode：100/100, Reward：777.337105666515
Finish Training!


In [ ]:
rewards_train_MSFT

[274.63149238009737,
 624.7199883835042,
 616.1498662576872,
 705.1165423727515,
 720.3606679146274,
 713.6365170497909,
 754.7630040835505,
 750.0279629652551,
 749.4825965706339,
 730.2911490806887,
 752.364996350266,
 752.4416095533674,
 744.3643003957287,
 760.3564636618992,
 746.6502411264506,
 784.9601662408476,
 775.6361804876741,
 770.04555652586,
 777.1611084002606,
 775.5594298001346,
 768.6652966132083,
 782.6080783098706,
 802.0496615492868,
 740.9734659786455,
 778.5464724226749,
 774.3261751228639,
 771.3133369826353,
 797.7131555282169,
 783.8388582672565,
 771.5692470785041,
 768.5382266213435,
 779.9802137817269,
 778.0511503635025,
 761.7482894529018,
 751.9912277198815,
 768.9286308687316,
 776.5417581924154,
 781.3588582522319,
 767.0846187609937,
 752.2763532588352,
 808.4676689417031,
 793.3727483862175,
 804.3738916960044,
 794.8741149032337,
 798.4945504142928,
 774.7925051511634,
 791.1692811327689,
 796.9261065946266,
 801.5163078574398,
 780.9465252369755,
 7

In [ ]:
ma_rewards_train_MSFT

[274.63149238009737,
 309.64034198043805,
 340.29129440816297,
 376.7738192046218,
 411.1325040756224,
 441.38290537303925,
 472.7209152440904,
 500.4516200162069,
 525.3547176716496,
 545.8483608125536,
 566.5000243663249,
 585.0941828850291,
 601.0211946360992,
 616.9547215386791,
 629.9242734974563,
 645.4278627717954,
 658.4486945433832,
 669.6083807416309,
 680.3636535074938,
 689.8832311367579,
 697.7614376844031,
 706.2461017469498,
 715.8264577271835,
 718.3411585523297,
 724.3616899393642,
 729.3581384577142,
 733.5536583102064,
 739.9696080320075,
 744.3565330555324,
 747.0778044578295,
 749.2238466741809,
 752.2994833849355,
 754.8746500827922,
 755.5620140198032,
 755.2049353898111,
 756.5773049377032,
 758.5737502631745,
 760.8522610620803,
 761.4754968319717,
 760.5555824746581,
 765.3467911213626,
 768.1493868478481,
 771.7718373326637,
 774.0820650897208,
 776.523313622178,
 776.3502327750766,
 777.8321376108458,
 779.741534509224,
 781.9190118440456,
 781.8217631833386

In [ ]:
# Testing Microsoft
env, agent = env_agent_config(MSFT_test, cfg, 'test',"MSFT")
stocks_test_MSFT, rewards_test_MSFT= test(cfg, env, agent)


Start Testing!
Environment：TradingSystem_v0, Algorithm：DQN, Device：cpu,Stock: MSFT
Stock：MSFT，Return：57.6
Finish Testing!


In [ ]:
stocks_test_MSFT

'MSFT'

In [ ]:
rewards_test_MSFT

57.55907878804892